# Get coordinates

In this notebook we'll be pulling the coordinates of every college and university in the United States from the Department of Education's [Integrated Postsecondary Education Data System (IPEDS)](https://nces.ed.gov/ipeds/) via [Collge Scorecard data](https://collegescorecard.ed.gov/data/).

Let's start by importing the necessary libraries.

In [35]:
# First step is we import pandas for data manipulation
import pandas as pd

Now let's import the data.

I previously downloaded the data from the College Scorecard website and saved it as a CSV file. I subsequently exported the file to [the parquet format](https://parquet.apache.org/). I did this so that I could upload the entire dataset to Github, as the original CSV file was too large.

In [36]:
full_college_df = pd.read_parquet('./data/Most-Recent-Cohorts-Institution.parquet')

full_college_df.head()

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,COUNT_WNE_MALE1_P8,MD_EARN_WNE_MALE1_P8,GT_THRESHOLD_P10,MD_EARN_WNE_INC1_P10,MD_EARN_WNE_INC2_P10,MD_EARN_WNE_INC3_P10,MD_EARN_WNE_INDEP1_P10,MD_EARN_WNE_INDEP0_P10,MD_EARN_WNE_MALE0_P10,MD_EARN_WNE_MALE1_P10
0,100654,100200.0,1002.0,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,www.aamu.edu/admissions-aid/tuition-fees/net-p...,...,834.0,36639.0,0.6044,34076.0,35597.0,43145.0,40299.0,35424.0,36050.0,36377.0
1,100663,105200.0,1052.0,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,https://www.uab.edu/,https://tcc.ruffalonl.com/University of Alabam...,...,1233.0,49652.0,0.7472,42254.0,49817.0,51571.0,48182.0,46435.0,42007.0,56164.0
2,100690,2503400.0,25034.0,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,www.amridgeuniversity.edu/,www2.amridgeuniversity.edu:9091/,...,78.0,50355.0,0.6286,36636.0,44836.0,nan,39040.0,nan,32311.0,49599.0
3,100706,105500.0,1055.0,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,www.uah.edu/,finaid.uah.edu/,...,891.0,57542.0,0.7769,49469.0,60533.0,57411.0,56884.0,53803.0,45170.0,66070.0
4,100724,100500.0,1005.0,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,www.alasu.edu/,www.alasu.edu/cost-aid/tuition-costs/net-price...,...,1077.0,32797.0,0.5178,30634.0,34533.0,38216.0,30602.0,32364.0,29836.0,35315.0


With the data loaded, let's now extract just want we want out of the dataset. This includes the following columns:
- INSTNM: Institution name
- CITY: City
- STABBREV: State abbreviation
- ZIP: Zip code
- LATITUDE: Latitude
- LONGITUDE: Longitude
- UNITID: Unit ID for institution
- OPEID: 8-digit OPE ID for institution
- INSTURL: Institution URL

In the following cell I also import a CSV file containing the state abbreviations and their full names. This will be used to convert the state abbreviations to their full names.

I'm keeping all the original variable names as they are.

In [37]:
# The columns we want to keep in our dataframe stored in a list
desired_columns = ['INSTNM', 'CITY', 'STABBR', 'ZIP', 'LATITUDE', 'LONGITUDE', 'UNITID', 'OPEID', 'INSTURL']

# Create a new dataframe with only the desired columns
college_coordinates_df = full_college_df[desired_columns]

# Import the state abbreviations csv
state_abbreviations_df = pd.read_csv('./data/state_abbreviations.csv')

# Merge the two dataframes
college_coordinates_df = college_coordinates_df.merge(state_abbreviations_df, left_on='STABBR', right_on='abbreviations')

# Drop abbreviations column
college_coordinates_df = college_coordinates_df.drop(columns=['abbreviations'])

# Reorder columns
college_coordinates_df = college_coordinates_df[['INSTNM', 'CITY', 'State ', 'STABBR', 'ZIP', 'LATITUDE', 'LONGITUDE', 'UNITID', 'OPEID', 'INSTURL']]

# Rename State to State
college_coordinates_df = college_coordinates_df.rename(columns={'State ': 'STATE'})

# Sort based on the INSTNM column
college_coordinates_df = college_coordinates_df.sort_values(by=['INSTNM'])

college_coordinates_df.head()

,INSTNM,CITY,STATE,STABBR,ZIP,LATITUDE,LONGITUDE,UNITID,OPEID,INSTURL
367,A Better U Beauty Barber Academy,Albuquerque,New Mexico,NM,87110,35.11063,-106.583765,493868,4283300.0,https://abuacademy.com/
3352,A T Still University of Health Sciences,Kirksville,Missouri,MO,63501,40.193648,-92.589183,177834,247700.0,www.atsu.edu/
1079,ABC Adult School,Cerritos,California,CA,90703-2801,33.878179,-118.070114,491464,4227000.0,https://www.abcadultschool.edu/
1171,ABC Adult School - Cabrillo Lane,Cerritos,California,CA,90703-7748,nan,nan,49146401,4227001.0,nan
5942,ABC Beauty Academy,Richardson,Texas,TX,75081,32.931698,-96.685333,459523,4168800.0,www.abcbeautyacademy.webs.com/


Great! Now we have all of our data. I'm going to save this dataframe as both a CSV and a parquet file.

In [38]:
college_coordinates_df.to_csv('./output/college_coordinates.csv', index=False)

college_coordinates_df.to_parquet('./output/college_coordinates.parquet', index=False)

The end. Hope this is useful!